Breaking RC4: A Cryptanalysis Project.

# part 1

In [ ]:
# here i am importing numpy, because i am storing probability distribution of a key in matrix like structure.
import numpy as np
# here i am importing Counter, so that i can count k_stream accurance.
from collections import Counter
# here i am importing random, to generate random key stream. 
import random
# here i am importing os, show that i can manage file path and directories.
import os

print("step-1, to break RC4 encryption :-")
print("work is in progress...")

# here is a  few constants that are given in the key.
# State array size 2^5 
N = 32  
# this is the total number of random keys which i have to generate.
NUM_KEYS = 2**24 
# this is the number of bytes in each k_stream.
k_stream_len = 6
# this is the fixed key of size which is 5 bytes.  
KEY_SIZE = 5  

# this is the directory and file where the probability dist of key stream is goin to store
key_outout_dict = './key_prob_disttribution'
os.makedirs(key_outout_dict, exist_ok=True)
pd_file = os.path.join(key_outout_dict, 'kkkey_probability_dist.txt')

# In this function i am generating a random key of the specified lenght, which is mentioned in the question.
def generate_random_key(len):
    key = []
    for i in range(len):
        # this will help me to generate generate a random integer which lies in between 0 and 31
        rand_number = random.randint(0, 31)
        # then adding this generated number into the key list
        key.append(rand_number)
    return key

# this the Key Scheduling Algorithm (KSA) which is taken from the provided pdf file.
def key_scheduling_algorithm(key):
    # below is the state array
    S = list(range(N))  
    j = 0
    l = len(key)
    
    # the below few line of code helps to permute S, based on the key values
    for i in range(N):
        a = (j + S[i] + key[i % l])
        j = a % N
        # here i am Swaping the values present at positions i and j
        temp = S[i]
        S[i] = S[j]
        S[j] = temp
    return S

# this is the Pseudo Random Generation Algorithm (PRGA) which helps to produce a k_stream
def pseudo_random_generation_algorithm(S, len):
    i = 0
    j = 0
    k_stream = []
    
    for _ in range(len):
        i = (i + 1) % N
        j = (j + S[i]) % N
        # Swap values at positions i and j
        S[i], S[j] = S[j], S[i]  
        # k_stream byte
        K = S[(S[i] + S[j]) % N] 
        # Keep values in the range [0, 31] 
        k_stream.append(K % 32)  
    return k_stream

# this helps me to track the frequency of key distribution in each position in the k_stream.
k_stream_dist = []
for _ in range(k_stream_len):
    # here i am using inbuild library to make count.
    k_stream_dist.append(Counter())

# this is used to Generate keys, run if KSA for key scheduling and PRGA for pseudo random generation, and updating key dist. 
for key_index in range(NUM_KEYS):
    # this helps me to generate a random of 5-byte keys
    key = generate_random_key(KEY_SIZE)  
    # this is for state space array
    S = key_scheduling_algorithm(key)
    # this is key stream generation
    k_stream = pseudo_random_generation_algorithm(S, k_stream_len)
    
    # this will helps me to update the distribution counters for each position in the k_stream
    for i in range(k_stream_len):
        k_stream_dist[i][k_stream[i]] =  k_stream_dist[i][k_stream[i]] + 1
    
    # this helps me to track tha program is working correctly by printing progress every 2^20 keys for tracking...
    if key_index % (2**20) == 0:
        print(f"Now processing {key_index}th keys out of {NUM_KEYS}")

# this helps me to calculate the probability distribution and save it into a text file
with open(pd_file, "w") as f:
    # this helps to write header row into the text file.
    f.write("Row/Col\t" + "\t".join([str(i + 1) for i in range(k_stream_len)]) + "\n")
    
    # now writing probability distribution for each value from 0 to 31) into the text file
    for val in range(N):
        row_data = []
        for i in range(k_stream_len):
            total = sum(k_stream_dist[i].values())
            probability = k_stream_dist[i][val] / total
            row_data.append(f"{probability:.8f}")
        f.write(f"{val}\t" + "\t".join(row_data) + "\n")

print("Probability distribution for a key is saved into this text file 'kkkey_probability_dist.txt'")


step-1, to break RC4 encryption :-
work is in progress...
Now processing 0th keys out of 16777216
Now processing 1048576th keys out of 16777216
Now processing 2097152th keys out of 16777216
Now processing 3145728th keys out of 16777216
Now processing 4194304th keys out of 16777216
Now processing 5242880th keys out of 16777216
Now processing 6291456th keys out of 16777216
Now processing 7340032th keys out of 16777216
Now processing 8388608th keys out of 16777216
Now processing 9437184th keys out of 16777216
Now processing 10485760th keys out of 16777216
Now processing 11534336th keys out of 16777216
Now processing 12582912th keys out of 16777216
Now processing 13631488th keys out of 16777216
Now processing 14680064th keys out of 16777216
Now processing 15728640th keys out of 16777216
Probability distribution for a key is saved into this text file 'kkkey_probability_dist.txt'


# part 2

In [2]:
# This function helps me to load all the cipher text files that are provided in the assignment from a folder,
# returning them as a list of lists of lists
# there are total of 4096 files, each with 4096 rows, each with 6 numbers of coloumns
def load_ciphr_txt_from_folder(fld_path):
    ciphr_txt = []
    for i in sorted(os.listdir(fld_path)):
        file_path = os.path.join(fld_path, i)
        with open(file_path, 'r') as f:
            fl_d = []
            for line in f:
                rows = list(map(int, line.strip().strip("[]").split(',')))
                fl_d.append(rows)
            ciphr_txt.append(fl_d)
    return ciphr_txt

# this function helps me to calculate probability distribution and save results into the text file.
def compute_and_save_prob_distribution(ciphr_txt, oput_fld):
    # the outer loop run over numbers the number from 0 to 9 because this are the all posible passcodes and converting it into ASCII
    for i in range(10):
        # this line helps me to convert `i`th numebr into ASCII code
        ascii_valu = ord(str(i))
        
        # this will helps me to create a output text files for the ASCII valu of `i` and the name the text file is look likes i.txt
        output_text_file_path = os.path.join(oput_fld, f"{i}.txt")
        
        with open(output_text_file_path, 'w') as output_file:
            # this is the header_names for the text file.
            header_name = ["rows/Col"]
            for j in range(1, 7):
                header_name.append(f"Col{j}")
            output_file.write("\t".join(header_name) + "\n")
            
            # now initializing a 2D array for storing probability distributions for all the rows 0-31 and all the columns 1-6
            prob_destri = list(map(lambda _: [0] * 6, range(32)))
            
            # this is the loop to iterate over colmns from 1 to 6
            for j in range(1, 7):
                # here i am initializing a vector of size 32, which will countes from value 0 to 31
                couting_vectors = [0] * 32
                
                # this is the nested loop helps to iterate over all the lists in all the cipher text files, in total 4096*4096 times it will run.
                for fl_d in ciphr_txt:
                    for rows in fl_d:
                        # computing XOR operation between ASCII value of `i` and the `j`th number in each rows of the text file.
                        # 31 has very segnificant role that helps the result is in the range 0 to 31
                        result = (ascii_valu ^ rows[j - 1]) & 31  
                        couting_vectors[result] = couting_vectors[result] + 1
                
                # now storing the probability distribution for each result value in the corresponding cell of a text file
                totl_valu = 4096 * 4096
                for valu in range(32):
                    prob_destri[valu][j - 1] = couting_vectors[valu] / totl_valu

            # this helps me to write the probability distributions into the text file and in the correct rows/column format.
            for rows_num in range(32):
                values = []
                for col in range(6):
                    values.append(f"{prob_destri[rows_num][col]:.8f}")
                output_file.write(f"{rows_num}\t" + "\t".join(values) + "\n")


def tood():
    print("Step-2, to break RC4 encryption")
    print("work is in progress...")


tood
# this line of code will get input from user, which is cipher text folder and output folder
cipher_txt_fld = input("Enter the path of the folder which contains cipher_text text files: ").strip()
oput_fld = input("Enter the path of the folder where output you want to store in text files: ").strip()

# now loading cipher texts from folder, there is cipher text file is present.
ciphr_txt = load_ciphr_txt_from_folder(cipher_txt_fld)
    
# now computing probability distributions and saving it into text file.
compute_and_save_prob_distribution(ciphr_txt, oput_fld)
print(f"The Probability distributions of 0 to 9 is saved in folder: {oput_fld}")



The Probability distributions of 0 to 9 is saved in folder: /home/anurag/ACN_Work/Cryptology/assign 2/final output is here


# part 3

In [3]:
# this function is to load matrix data from a text file by avoding row and column header.
def loading_matrix(text_file_path):
    with open(text_file_path, 'r') as f:
        # this is to avoid the header row from the text file.
        next(f)
        # this is to load numerical data only and ignoring row labels from the text file.
        data = []
        for line in f:
            values = line.strip().split()[1:]  # this is to skiping the first element
            converted_valu = []
            for value in values:
                converted_valu.append(float(value))
            data.append(converted_valu)
            matrix = np.array(data)
        return matrix

# this function helps me to find the file with minimum absolute difference for each column, form 1 to 6
def find_min_diiff(prob_d_path, comparision_between_the_folder, out_put_path):
    # this line helps me to load the main probability distribution matrix form the text file.
    probability_matrics = loading_matrix(prob_d_path)
    
    # here it will store the text file with minimum difference value for each column as a passcode.
    min_files = []
    
    # this outer loop will iterate over a each of the six column of the text file
    for col in range(6):
        # in this, i am initializing a large value
        min_diff = float('inf')  
        # this variable is to store the file index with the minimum difference
        min_file = None  

        # this inner loop will iterate over the each file for comparison, files from 0.txt to 9.txt
        for i in range(10):
            comp_file_path = os.path.join(comparision_between_the_folder, f"{i}.txt")
            if not os.path.isfile(comp_file_path):
                print(f"Warning!!!: {comp_file_path} does not exist. So, skipping this text file.")
                continue
            
            # in this variable i am loading the comparison file matrix
            comp_matrix = loading_matrix(comp_file_path)
            
            # nOw here, main processing is started 
            # so here i am calculating the cumulative absolute difference across all the rows for that perticular column
            ttl_abs_diff = 0  # this will tracking the cumulative difference for the current column.
            for rw in range(32):
                row_diff = abs(probability_matrics[rw, col] - comp_matrix[rw, col])  # it will computes row-wise difference
                ttl_abs_diff = ttl_abs_diff + row_diff 

            # this line of code helps me to update minimum difference and the text file index, if the current difference is smaller.
            if ttl_abs_diff < min_diff:
                min_diff = ttl_abs_diff
                min_file = i
        
        # now appending the text file index with the minimum difference, for the current columns.
        min_files.append(min_file)
   
    # i am storing the passcode in one text file as a result. name of the text file is passcode.txt
    with open(out_put_path, 'w') as f:
        passcode = ''.join(map(str, min_files))
        f.write(passcode + "\n")
    print(f"The 6-digit passcode is saved in {out_put_path}: {passcode}")

def doooo():
    print("Step-3, to break RC4 encryption")
    print("work is in progress...")

# this is the main function to execute the entire program

doooo
probability_distribution_file = input("Please enter the path of the keys probability distributions text file: ").strip()
comparision_between_the_folder = input("Please enter the path of the folder, which contains the comparison files (0.txt to 9.txt): ").strip()
oput_file = os.path.join(comparision_between_the_folder, '6_digit_passcode.txt')
find_min_diiff(probability_distribution_file, comparision_between_the_folder, oput_file)


The 6-digit passcode is saved in /home/anurag/ACN_Work/Cryptology/assign 2/final output is here/6_digit_passcode.txt: 475103
